# 1. SageMaker Overview
SageMaker makes extensive use of Docker containers to allow users to train and deploy algorithms. Containers allow developers and data scientists to package software into standardized units that run consistently on any platform that supports Docker. Containerization packages code, runtime, system tools, system libraries and settings all in the same place, isolating it from its surroundings, and insuring a consistent runtime regardless of where it is being run.

When you develop a model in Amazon SageMaker, you can provide separate Docker images for the training code and the inference code, or you can combine them into a single Docker image. In this blog post, we build a single image to support both training and hosting.

## 1. Anatomy of a SageMaker Container

<img src="images/scikit-sagemaker-1.gif">

An example of a container folder can be found <a href="https://github.com/awslabs/amazon-sagemaker-examples/tree/master/advanced_functionality/scikit_bring_your_own/container">here</a>. If we take a look at the repo we will see the following contents:

---


``` bash
container/
    decision_tress/
        nginx.conf
        predictor.py
        serve
        train
        wsgi.py
    Dockerfile
```

---

Let's break this down. First we have the `Dockerfile`, which is used to build the container. Inside the docker file we can specify our requirements and dependencies that must be installed inside of the container. This is similar to the process of making sure that you have the required packages and dependencies locally in order to run certain software-we are now just ensuring that that same process is done for our container. A few notes on the Dockerfile:
* `apt-get` is similar to homebrew in the sense that it is a free software user interface that works with core libraries to handle the installation/removal of software on linux and ubuntu. 
* there are two other lines that may cause potential confusion:
```
ENV PATH="/opt/program:${PATH}"
COPY decision_trees /opt/program
WORKDIR /opt/program
```
We update `PATH` so that the train and serve programs are found when the container is invoked.These have been added so that when the container is built, it copies everything in the `decision_trees` folder and places it in `/opt/program`, which will be on the docker image. `WORKDIR` is like when you jump into the shell and you start the home directory, `WORKDIR` is doing the same thing. 

### 1.1 Container Startup
When you actually run a docker image, it will either exectute the program defined in `ENTRYPOINT`, or it will look for a program whose name is passed in as an arugment. If an `ENTRYPOINT` is defined, then the following command will run the default startup program:

`docker run <image>`

On the other hand, if no entry point is defined, a specified program can be executed with the following command:

`docker run <image> <startup program>`

For an image built for only training or hosting, but not both, there is no need to build unused functionalities in the program folder. If you are building an image for training only, you only need to have a train file in the program folder. The train can also be set as the ENTRYPOINT, and the image starts with the train program by default for training service.

Similarly, if you are building an image for hosting only, files to be included in the program folder are nginx.conf, predictor.py, serve, and wsgi.py. The serve can be set as the ENTRYPOINT, and the image would start the serve program by default.

In this blog post we are building a single image for both training and hosting purpose, so we don’t define a default startup program as an ENTRYPOINT. Instead, Amazon SageMaker runs the image by using one of the two following commands. Then the container starts programs for training or hosting service accordingly.

`docker run <image> train`

`docker run <image> serve`